In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
creditcardinfo = pd.read_csv("creditcard.csv")
print(creditcardinfo.head())

   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V27       V28 

In [10]:
creditcardinfo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [11]:
creditcardinfo['Class'].value_counts()#we are checking to see the number of fraudulent transactions and legit ones

0    284315
1       492
Name: Class, dtype: int64

In [12]:
#the dataset is highly unbalanced there is not enough fruadulaent data for the machine to recogognise fraudlent data
# we need to analyse the data seperatly
#in the dataset in the class column 0 = nonfraud, 1 = fraud
nonfraud = creditcardinfo[creditcardinfo.Class == 0]
fraud = creditcardinfo[creditcardinfo.Class == 1]
#to see the statistics of the average amounts spent on these transactions for class =0, i will output below.
nonfraud.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [13]:
#for class = 1(fraud)
fraud.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [14]:
#can see the average spent on fraudelent data is higher than nonfruad on most statistaical measures and can see them compared
creditcardinfo.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [15]:
#Build a sample dataset that has similar distrubition of normal and fraudulant transaction this helps to balance the dataset
#because we only have 492 fraudulaent we cant add more values to that column so we take 492 random transactions from the non fruadulent
nonfraudsample = nonfraud.sample(n = 492)

In [16]:
#we now are combining the 2 data sets fraud and random non fraud transactions
newset = pd.concat([nonfraudsample, fraud], axis =0)
newset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
188155,127842.0,0.041100,0.906730,0.279707,-0.615871,0.542964,-0.989756,1.033941,-0.215552,-0.144730,...,-0.252105,-0.504169,0.055749,-0.071544,-0.472108,0.145528,0.255288,0.097573,1.29,0
95185,65198.0,-0.322616,0.969557,1.498098,0.716105,0.452123,-0.395224,0.866644,-0.340203,-0.371810,...,-0.013160,0.145323,-0.168858,-0.129746,-0.200451,-0.423443,-0.127792,-0.086902,2.50,0
171349,120601.0,0.052875,-0.336768,0.260870,-0.361526,0.674344,0.504039,1.526389,-1.154178,1.189326,...,0.135885,1.457088,0.028438,0.184537,-2.234617,-0.065674,-0.558332,-0.410481,190.00,0
10637,17876.0,1.076256,0.250711,0.502916,1.246392,0.136684,0.438216,-0.216999,0.155620,1.038624,...,0.034257,0.382551,-0.060216,-0.337349,0.459101,-0.278657,0.016302,0.000954,19.99,0
162005,114737.0,-1.577976,2.366350,-0.403456,-0.484032,-0.156529,-1.322586,0.308892,-2.726672,0.712265,...,2.617318,0.730191,0.370096,0.940808,-0.604979,-0.244295,1.041051,0.597189,0.89,0


In [17]:
newset['Class'].value_counts()
#you can see the dataset is balanced with an equal amount of fraudulent transactions to non fraud

0    492
1    492
Name: Class, dtype: int64

In [18]:
#finding features and targets in the data 
X = newset.drop(columns = 'Class', axis =1)
Y = newset['Class']
print(X)

            Time        V1        V2        V3        V4        V5        V6  \
188155  127842.0  0.041100  0.906730  0.279707 -0.615871  0.542964 -0.989756   
95185    65198.0 -0.322616  0.969557  1.498098  0.716105  0.452123 -0.395224   
171349  120601.0  0.052875 -0.336768  0.260870 -0.361526  0.674344  0.504039   
10637    17876.0  1.076256  0.250711  0.502916  1.246392  0.136684  0.438216   
162005  114737.0 -1.577976  2.366350 -0.403456 -0.484032 -0.156529 -1.322586   
...          ...       ...       ...       ...       ...       ...       ...   
279863  169142.0 -1.927883  1.125653 -4.518331  1.749293 -1.566487 -2.010494   
280143  169347.0  1.378559  1.289381 -5.004247  1.411850  0.442581 -1.326536   
280149  169351.0 -0.676143  1.126366 -2.213700  0.468308 -1.120541 -0.003346   
281144  169966.0 -3.113832  0.585864 -5.399730  1.817092 -0.840618 -2.943548   
281674  170348.0  1.991976  0.158476 -2.583441  0.408670  1.151147 -0.096695   

              V7        V8        V9  .

In [19]:
print(Y)

188155    0
95185     0
171349    0
10637     0
162005    0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: Class, Length: 984, dtype: int64


In [20]:
#we can split the data into train and test datasets now
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify =Y, random_state =2)
print(X.shape, X_train.shape, X_test.shape)

(984, 30) (787, 30) (197, 30)
